In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [2]:
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'

In [3]:
transactions_full = pd.read_csv('./data/transactions_train.csv', parse_dates=["TX_TS"])
# transactions_train_og = transactions_train_og.set_index("TX_ID")

merchants_full = pd.read_csv('./data/merchants.csv')

terminals_full = pd.read_csv('./data/terminals.csv')

customers_full = pd.read_csv('./data/customers.csv')

C:\Users\joshu\AppData\Local\Temp\ipykernel_11716\1249260220.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_full = pd.read_csv('./data/transactions_train.csv', parse_dates=["TX_TS"])


In [4]:
transactions_full = pd.merge(transactions_full,merchants_full,on='MERCHANT_ID')

In [5]:
useless_columns = ['CARD_EXPIRY_DATE','TX_ID','CARD_DATA','LEGAL_NAME','FOUNDATION_DATE','ACTIVE_FROM','TRADING_FROM']
transactions_full.drop(useless_columns,axis=1,inplace=True)

In [6]:
transactions_full["TX_TS"] = pd.to_datetime(transactions_full["TX_TS"],format='%Y-%m-%d')

In [7]:
transactions_full.dropna(subset=['ACQUIRER_ID','CARDHOLDER_AUTH_METHOD'], inplace=True)

In [8]:
transactions_full['ANNUAL_TURNOVER_CASH'] = transactions_full['ANNUAL_TURNOVER'] - transactions_full['ANNUAL_TURNOVER_CARD']

bins = [0, 6, 12, 18, 24]
labels = ['00:00-05:59', '06:00-11:59', '12:00-17:59', '18:00-23:59']
transactions_full['Time_of_day'] = pd.cut(transactions_full.TX_TS.dt.hour, bins, labels=labels, right=False)

bins = np.arange(12)
transactions_full['Month'] = transactions_full['TX_TS'].apply(lambda x: x.month)

transactions_full['Day_of_week'] = transactions_full['TX_TS'].apply(lambda x: x.weekday())

In [9]:
transactions_full.drop(['TX_TS'],axis=1,inplace=True)

In [10]:
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == 'True',['IS_RECURRING_TRANSACTION']] = 1
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == True,['IS_RECURRING_TRANSACTION']] = 1
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == 'False',['IS_RECURRING_TRANSACTION']] = 0
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == False,['IS_RECURRING_TRANSACTION']] = 0

transactions_full['IS_RECURRING_TRANSACTION'] = transactions_full['IS_RECURRING_TRANSACTION'].apply(pd.to_numeric)

In [11]:
one_hot_encoded_col = ['CARD_BRAND','TRANSACTION_TYPE','TRANSACTION_STATUS','FAILURE_CODE','FAILURE_REASON','TRANSACTION_CURRENCY','CARD_COUNTRY_CODE','ACQUIRER_ID','CARDHOLDER_AUTH_METHOD','BUSINESS_TYPE','OUTLET_TYPE','Time_of_day']
transactions_full = pd.get_dummies(transactions_full, columns = one_hot_encoded_col)

In [12]:
ordinal_encoder = OrdinalEncoder()
transactions_full['MERCHANT_ID'] = ordinal_encoder.fit_transform(transactions_full['MERCHANT_ID'].to_numpy().reshape(-1, 1))

In [13]:
scaler = MinMaxScaler()
scaler.fit(transactions_full)

MinMaxScaler()

In [14]:
transactions_full_normalised = pd.DataFrame(scaler.transform(transactions_full), index=transactions_full.index, columns=transactions_full.columns)

In [15]:
transactions_full_normalised.describe()

,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,TRANSACTION_GOODS_AND_SERVICES_AMOUNT,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,MCC_CODE,TAX_EXCEMPT_INDICATOR,...,BUSINESS_TYPE_Limited Liability Company (LLC),BUSINESS_TYPE_S Corporations,BUSINESS_TYPE_Sole Proprietorships,OUTLET_TYPE_Ecommerce,OUTLET_TYPE_Face to Face,OUTLET_TYPE_Face to Face and Ecommerce,Time_of_day_00:00-05:59,Time_of_day_06:00-11:59,Time_of_day_12:00-17:59,Time_of_day_18:00-23:59
count,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,...,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06,1.066439e+06
mean,4.993912e-01,4.997569e-01,8.736308e-02,2.584677e-02,8.696733e-02,3.005191e-03,5.020969e-01,9.548601e-02,6.091865e-01,1.018286e-01,...,6.140783e-01,3.401695e-02,2.295199e-01,2.108409e-01,4.610512e-01,3.281078e-01,2.510308e-01,2.494301e-01,2.494142e-01,2.501249e-01
std,2.885753e-01,2.873206e-01,4.827118e-02,1.586781e-01,4.809948e-02,1.853885e-02,2.854943e-01,2.938852e-01,1.698029e-01,3.024229e-01,...,4.868125e-01,1.812728e-01,4.205243e-01,4.079059e-01,4.984809e-01,4.695245e-01,4.336064e-01,4.326834e-01,4.326741e-01,4.330850e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.519763e-01,2.515138e-01,4.710199e-02,0.000000e+00,4.684582e-02,0.000000e+00,2.558063e-01,0.000000e+00,5.103171e-01,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.965680e-01,4.959148e-01,8.643628e-02,0.000000e+00,8.597352e-02,0.000000e+00,5.025144e-01,0.000000e+00,5.685274e-01,0.000000e+00,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,7.480796e-01,7.435887e-01,1.255888e-01,0.000000e+00,1.249773e-01,0.000000e+00,7.496526e-01,0.000000e+00,7.492398e-01,0.000000e+00,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [16]:
# Divide data into training and validation subsets
y = transactions_full_normalised['TX_FRAUD']
X = transactions_full_normalised.drop(['TX_FRAUD'],axis = 1)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=42)

In [ ]:
X_train_full.to_csv('./data/feature_set_normalised/X_train_full.csv',index=False)
X_valid_full.to_csv('./data/feature_set_normalised/X_valid_full.csv',index=False)
y_train.to_csv('./data/feature_set_normalised/y_train.csv',index=False)
y_valid.to_csv('./data/feature_set_normalised/y_valid.csv',index=False)

PCA

In [53]:
for num in np.arange(20,80,5):
  pca = PCA(n_components=num)
  pca.fit(transactions_full_normalised.drop(['TX_FRAUD'],axis = 1))
  print(F'{num} {sum(pca.explained_variance_ratio_)}')

20 0.7069799195409826
25 0.7754454871852794
30 0.8349627443494321
35 0.8819133478352716
40 0.9161655653004763
45 0.9453911618560551
50 0.9672422053569516
55 0.9804126662335245
60 0.9882880591235035
65 0.9934199996758752
70 0.9966921289239031
75 0.9987989688890778


In [54]:
pca = PCA(n_components=55)
pca.fit(X)

PCA(n_components=55)

In [55]:
sum(pca.explained_variance_ratio_)

0.9804150195877338

In [56]:
# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component
# LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]

initial_feature_names = pca.feature_names_in_
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(dic.items())

In [57]:
subset_pca_data = transactions_full_normalised[most_important_names]

In [58]:
subset_pca_data

,CARD_BRAND_Visa,CARD_COUNTRY_CODE_US,OUTLET_TYPE_Face to Face,BUSINESS_TYPE_Limited Liability Company (LLC),ACQUIRER_ID_ACQ2,CARDHOLDER_AUTH_METHOD_Online PIN,TRANSACTION_CURRENCY_GBP,ACQUIRER_ID_ACQ1,Time_of_day_00:00-05:59,Time_of_day_18:00-23:59,...,TRANSACTION_CURRENCY_JPY,PAYMENT_PERCENTAGE_MOTO,TRANSACTION_STATUS_Captured,MCC_CODE,CARD_COUNTRY_CODE_DK,CARD_COUNTRY_CODE_DK,DELIVERY_OVER_TWO_WEEKS_PERCENTAGE,TRANSACTION_STATUS_Authorized,CARD_COUNTRY_CODE_RO,ACQUIRER_ID_ACQ5
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.628910,0.0,0.0,0.780488,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.628910,0.0,0.0,0.780488,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.628910,0.0,0.0,0.780488,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.628910,0.0,0.0,0.780488,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.628910,0.0,0.0,0.780488,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066435,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.505756,0.0,0.0,0.536585,0.0,0.0,0.0
1066436,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.520091,0.0,0.0,0.414634,1.0,0.0,0.0
1066437,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.520308,0.0,0.0,0.512195,0.0,0.0,0.0
1066438,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.812120,0.0,0.0,0.560976,0.0,0.0,0.0


In [59]:
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(subset_pca_data, y, train_size=0.7, test_size=0.3,random_state=42)

In [60]:
X_train_full.to_csv('./data/feature_set_normalized_pca/X_train_full.csv',index=False)
X_valid_full.to_csv('./data/feature_set_normalized_pca/X_valid_full.csv',index=False)
y_train.to_csv('./data/feature_set_normalized_pca/y_train.csv',index=False)
y_valid.to_csv('./data/feature_set_normalized_pca/y_valid.csv',index=False)